# Importing the libraries

# New Section

# Importing the dataset

In [ ]:
!git clone https://github.com/mandar196/Hate_Speech_Detection-NLP.git

Cloning into 'Hate_Speech_Detection-NLP'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:
!gdown --id 1OPiQvG-AAtFNb9jJQnduvDZjz-CrZjxn #training data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#to data preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#NLP tools
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

#train split and fit models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

#model selection
from sklearn.metrics import confusion_matrix, accuracy_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
dataset = pd.read_csv('/content/final_dataset.txt')
dataset.head()

,Tweet Id,Tweets,User Id,Screen Name,Class
0,319399851215433000,Oh yeah Colin! Smash those girls! :D #MKR,949380854,CreatrixKali,sexism
1,320817818222358000,It's insane they keep bringing people back. Wh...,297877558,quincepoacher,none
2,324114200450437000,@berkeley_eagle #MKR this shit show has more ...,272704749,MarkTramby,none
3,326286656854454000,*sigh* oh Colin 😍 #MKR,71416635,BinnyD,none
4,381988216292655000,"RT @brian_day15: I swear, I'm not sexist, but ...",930620467,YesYoureSexist,sexism


In [3]:
print(dataset['Class'].unique())

['sexism' 'none' 'racism']


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11325 entries, 0 to 11324
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Tweet Id     11325 non-null  int64 
 1   Tweets       11325 non-null  object
 2   User Id      11325 non-null  int64 
 3   Screen Name  11325 non-null  object
 4   Class        11325 non-null  object
dtypes: int64(2), object(3)
memory usage: 442.5+ KB


In [5]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Tweet Id,11325.0,5.535716e+17,4.374880e+16,3.193999e+17,5.635700e+17,5.714739e+17,5.755965e+17,6.183671e+17
User Id,11325.0,7.157336e+08,9.438027e+08,2.105021e+06,1.385734e+07,2.388825e+08,9.306205e+08,3.079113e+09


In [6]:
dataset['Class'] = dataset['Class'].replace({'sexism': 0})
dataset['Class'] = dataset['Class'].replace({'none': 1})
dataset['Class'] = dataset['Class'].replace({'racism': 0})

In [7]:
dicti = { 'Class': 'class','Tweets': 'tweet'}
dataset.rename(columns=dicti,
          inplace=True)

In [8]:
dt_trasformed = dataset[['class', 'tweet']]
y = dt_trasformed.iloc[:, :-1].values
print(y)

[[0]
 [1]
 [1]
 ...
 [1]
 [0]
 [0]]


### Encoding the Dependent Variable

In [ ]:
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
# y = np.array(ct.fit_transform(y))

In [ ]:
print(y)

[[0]
 [1]
 [1]
 ...
 [1]
 [0]
 [0]]


I separated this y in two variables that we will use to fit hate speech models and offensive speech models

In [ ]:
# y_df = pd.DataFrame(y)
# y_hate = np.array(y_df[0])
# y_offensive = np.array(y_df[1])
# y_neither = np.array(y_df[2])
# print(sum(y_neither==2))

0


In [9]:
#extra
xy = np.ravel(y,order='C')

In [ ]:
# print(y_hate)
# print(y_offensive)

[0. 1. 1. ... 1. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


## Cleaning the texts

In [10]:
corpus = []
for i in range(0, 11325):
  review = re.sub('[^a-zA-Z]', ' ', dt_trasformed['tweet'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [11]:
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(corpus).toarray()

## Splitting the dataset into the Training set and Test set

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, xy, test_size = 0.30, random_state = 0,shuffle=True)

In [ ]:
# dfd = pd.DataFrame(y_test)
# dfd.to_csv('ML_WS_ytest.csv')
# # print(type(data))
# dfdd = pd.DataFrame(X_test)
# dfdd.to_csv('ML_WS_Xtest.csv')

## Finding the best models to predict hate speech

**Naive Bayes**

In [13]:
classifier_np = GaussianNB()
classifier_np.fit(X_train, y_train)

GaussianNB()

**Decision Tree**

In [14]:
classifier_dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_dt.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

**KNN**

In [15]:
classifier_knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier_knn.fit(X_train, y_train)

KNeighborsClassifier()

**Logistic Regression**

In [16]:
classifier_lr = LogisticRegression(random_state = 0)
classifier_lr.fit(X_train, y_train)

LogisticRegression(random_state=0)

**Random Forest**

In [17]:
classifier_rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier_rf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

SVM Classifier

In [18]:
classifier_svm = svm.SVC()
classifier_svm.fit(X_train, y_train)

SVC()

XGBoost Classifier

In [19]:
classifier_xgb= XGBClassifier()
classifier_xgb.fit(X_train, y_train)

XGBClassifier()

# Making the Confusion Matrix for each model

In [20]:
from sklearn.metrics import classification_report

In [21]:
#Naive Bayes
y_pred_np = classifier_np.predict(X_test)
cm = confusion_matrix(y_test, y_pred_np)
print(classification_report(y_test,y_pred_np))
print(y_pred_np)
print(cm)


              precision    recall  f1-score   support

           0       0.34      0.87      0.49       898
           1       0.89      0.39      0.54      2500

    accuracy                           0.52      3398
   macro avg       0.62      0.63      0.51      3398
weighted avg       0.75      0.52      0.53      3398

[0 0 0 ... 1 0 0]
[[ 781  117]
 [1527  973]]


In [22]:
#KNN
y_pred_knn = classifier_knn.predict(X_test)
cm = confusion_matrix(y_test, y_pred_knn)
print(classification_report(y_test,y_pred_knn))
print(cm)

              precision    recall  f1-score   support

           0       0.79      0.36      0.49       898
           1       0.81      0.97      0.88      2500

    accuracy                           0.81      3398
   macro avg       0.80      0.66      0.69      3398
weighted avg       0.80      0.81      0.78      3398

[[ 319  579]
 [  83 2417]]


In [23]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_test)
cm = confusion_matrix(y_test, y_pred_xgb)
print(classification_report(y_test,y_pred_xgb))
print(cm)

              precision    recall  f1-score   support

           0       0.89      0.47      0.61       898
           1       0.84      0.98      0.90      2500

    accuracy                           0.84      3398
   macro avg       0.86      0.72      0.76      3398
weighted avg       0.85      0.84      0.83      3398

[[ 419  479]
 [  52 2448]]


In [24]:
#SVM
y_pred_svm = classifier_svm.predict(X_test)
cm = confusion_matrix(y_test, y_pred_svm)
print(classification_report(y_test,y_pred_svm))
print(cm)

              precision    recall  f1-score   support

           0       0.84      0.56      0.67       898
           1       0.86      0.96      0.91      2500

    accuracy                           0.86      3398
   macro avg       0.85      0.76      0.79      3398
weighted avg       0.85      0.86      0.84      3398

[[ 499  399]
 [  92 2408]]


In [25]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_test)
cm = confusion_matrix(y_test, y_pred_lr)
print(classification_report(y_test,y_pred_lr))
print(cm)

              precision    recall  f1-score   support

           0       0.81      0.63      0.71       898
           1       0.88      0.95      0.91      2500

    accuracy                           0.86      3398
   macro avg       0.84      0.79      0.81      3398
weighted avg       0.86      0.86      0.86      3398

[[ 568  330]
 [ 133 2367]]


In [26]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_test)
cm = confusion_matrix(y_test, y_pred_dt)
print(classification_report(y_test,y_pred_dt))
print(cm)


              precision    recall  f1-score   support

           0       0.66      0.68      0.67       898
           1       0.88      0.88      0.88      2500

    accuracy                           0.82      3398
   macro avg       0.77      0.78      0.78      3398
weighted avg       0.83      0.82      0.83      3398

[[ 612  286]
 [ 311 2189]]


In [27]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred_rf)
print(classification_report(y_test,y_pred_rf))
print(cm)

              precision    recall  f1-score   support

           0       0.76      0.66      0.70       898
           1       0.88      0.92      0.90      2500

    accuracy                           0.85      3398
   macro avg       0.82      0.79      0.80      3398
weighted avg       0.85      0.85      0.85      3398

[[ 591  307]
 [ 189 2311]]


In [28]:
rf_score = accuracy_score(y_test, y_pred_rf)
knn_score = accuracy_score(y_test, y_pred_knn)
svm_score = accuracy_score(y_test, y_pred_svm)
xgb_score = accuracy_score(y_test, y_pred_xgb)
lr_score = accuracy_score(y_test, y_pred_lr)
dt_score = accuracy_score(y_test, y_pred_dt)
np_score = accuracy_score(y_test, y_pred_np)

print('Random Forest Accuracy: ', str(rf_score))
print('K Nearest Neighbours Accuracy: ', str(knn_score))
print('Support Vector Machine Accuracy: ', str(svm_score))
print('XGBoost Classifier Accuracy: ', str(xgb_score))
print('Logistic Regression Accuracy: ',str(lr_score))
print('Decision Tree Accuracy: ', str(dt_score))
print('Naive Bayes Accuracy: ', str(np_score))

Random Forest Accuracy:  0.8540317834020011
K Nearest Neighbours Accuracy:  0.8051795173631549
Support Vector Machine Accuracy:  0.855503237198352
XGBoost Classifier Accuracy:  0.8437316068275457
Logistic Regression Accuracy:  0.8637433784579164
Decision Tree Accuracy:  0.8243084167157151
Naive Bayes Accuracy:  0.5161859917598587


So XGBoost Classifier looks better to predict hate speech based on this dataset. It's important to emphasize Random Forest,KNN and Decision Tree had great results as well. This Dataset looks like a product of artificial intelligence to classify hate and offensive speech.

In [29]:
#Now finally saving the model with XGB Classifier.
import joblib
joblib.dump(classifier_xgb, 'hatespeech.pkl')

['hatespeech.pkl']

In [30]:
from sklearn.metrics import precision_recall_fscore_support

In [31]:
score_rf=precision_recall_fscore_support(y_test,y_pred_rf,average='weighted')
print(score_rf)

(0.8496895152072939, 0.8540317834020011, 0.8505821216132166, None)


In [32]:
score_knn=precision_recall_fscore_support(y_test,y_pred_knn,average='weighted')
print(score_knn)

(0.803251279442217, 0.8051795173631549, 0.7768047898976187, None)


In [33]:
score_svm=precision_recall_fscore_support(y_test,y_pred_svm,average='weighted')
print(score_svm)

(0.8542814060814637, 0.855503237198352, 0.8447865973412121, None)


In [34]:
score_xgb=precision_recall_fscore_support(y_test,y_pred_xgb,average='weighted')
print(score_xgb)

(0.8504225372525037, 0.8437316068275457, 0.8255086939480222, None)


In [35]:
score_lr=precision_recall_fscore_support(y_test,y_pred_lr,average='weighted')
print(score_lr)

(0.8598375384851972, 0.8637433784579164, 0.8579323389860439, None)


In [36]:
score_dt=precision_recall_fscore_support(y_test,y_pred_dt,average='weighted')
print(score_dt)

(0.8259372365350198, 0.8243084167157151, 0.8250729908958947, None)


In [37]:
score_np=precision_recall_fscore_support(y_test,y_pred_np,average='weighted')
print(score_np)

(0.7461812810862835, 0.5161859917598587, 0.5275659538009804, None)


# Testing

In [38]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Test on Large Facebook

In [47]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB!_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB!_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [48]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='micro')
print(score_np)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='macro')
print(score_np)

              precision    recall  f1-score   support

           0       0.35      0.39      0.37       929
           1       0.62      0.58      0.60      1621

    accuracy                           0.51      2550
   macro avg       0.49      0.48      0.48      2550
weighted avg       0.52      0.51      0.52      2550

[[362 567]
 [682 939]]
(0.5226775326759973, 0.5101960784313726, 0.5154640199488113, None)
(0.5101960784313726, 0.5101960784313726, 0.5101960784313726, None)
(0.4851246355573873, 0.4844691810726943, 0.48376475446996225, None)


In [49]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='micro')
print(score_knn)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='macro')
print(score_knn)

              precision    recall  f1-score   support

           0       0.21      0.01      0.01       929
           1       0.63      0.99      0.77      1621

    accuracy                           0.63      2550
   macro avg       0.42      0.50      0.39      2550
weighted avg       0.48      0.63      0.50      2550

[[   6  923]
 [  23 1598]]
(0.47832117448054334, 0.6290196078431373, 0.4950639920467199, None)
(0.6290196078431373, 0.6290196078431373, 0.6290196078431373, None)
(0.4203859989878127, 0.49613489261303306, 0.39206700745658557, None)


In [50]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='micro')
print(score_xgb)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='macro')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.56      0.01      0.01       929
           1       0.64      1.00      0.78      1621

    accuracy                           0.64      2550
   macro avg       0.60      0.50      0.39      2550
weighted avg       0.61      0.64      0.50      2550

[[   5  924]
 [   4 1617]]
(0.6069241433947316, 0.636078431372549, 0.49783142163748656, None)
(0.636078431372549, 0.636078431372549, 0.636078431372549, None)
(0.595959595959596, 0.5014572593695901, 0.39384562735850503, None)


In [51]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='micro')
print(score_svm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='macro')
print(score_svm)

              precision    recall  f1-score   support

           0       0.24      0.01      0.02       929
           1       0.63      0.98      0.77      1621

    accuracy                           0.63      2550
   macro avg       0.44      0.50      0.39      2550
weighted avg       0.49      0.63      0.50      2550

[[   9  920]
 [  29 1592]]
(0.4891560673884036, 0.6278431372549019, 0.49650441964405323, None)
(0.6278431372549019, 0.6278431372549019, 0.6278431372549019, None)
(0.43530003352329866, 0.4958988225716162, 0.3944994896926421, None)


In [52]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='micro')
print(score_lr)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='macro')
print(score_lr)

              precision    recall  f1-score   support

           0       0.44      0.09      0.14       929
           1       0.64      0.94      0.76      1621

    accuracy                           0.63      2550
   macro avg       0.54      0.51      0.45      2550
weighted avg       0.57      0.63      0.54      2550

[[  80  849]
 [ 101 1520]]
(0.5688922642559376, 0.6274509803921569, 0.5368460897872663, None)
(0.6274509803921569, 0.6274509803921569, 0.6274509803921569, None)
(0.5418049436902532, 0.5119034417086291, 0.45302445302445304, None)


In [53]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='micro')
print(score_dt)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='macro')
print(score_dt)

              precision    recall  f1-score   support

           0       0.35      0.22      0.27       929
           1       0.63      0.76      0.69      1621

    accuracy                           0.57      2550
   macro avg       0.49      0.49      0.48      2550
weighted avg       0.53      0.57      0.54      2550

[[ 206  723]
 [ 381 1240]]
(0.52940539779487, 0.5670588235294117, 0.5388809422185761, None)
(0.5670588235294117, 0.5670588235294117, 0.5670588235294117, None)
(0.49131158111606454, 0.493351855922237, 0.4818660478703355, None)


In [54]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='micro')
print(score_rf)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='macro')
print(score_rf)

              precision    recall  f1-score   support

           0       0.35      0.17      0.23       929
           1       0.63      0.82      0.71      1621

    accuracy                           0.58      2550
   macro avg       0.49      0.50      0.47      2550
weighted avg       0.53      0.58      0.54      2550

[[ 160  769]
 [ 294 1327]]
(0.5308522763613297, 0.5831372549019608, 0.5381858977235106, None)
(0.5831372549019608, 0.5831372549019608, 0.5831372549019608, None)
(0.49276679725594374, 0.4954293386917802, 0.4726988678972208, None)


## Test on Facebook

In [39]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [40]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.50      0.27      0.35        59
           1       0.63      0.82      0.71        90

    accuracy                           0.60       149
   macro avg       0.57      0.55      0.53       149
weighted avg       0.58      0.60      0.57       149

[[16 43]
 [16 74]]
(0.5800206504904492, 0.6040268456375839, 0.5711079116389883, None)


In [41]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.39      0.12      0.18        59
           1       0.60      0.88      0.71        90

    accuracy                           0.58       149
   macro avg       0.50      0.50      0.45       149
weighted avg       0.52      0.58      0.50       149

[[ 7 52]
 [11 79]]
(0.5182500241929516, 0.5771812080536913, 0.503833316308642, None)


In [42]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.33      0.02      0.03        59
           1       0.60      0.98      0.75        90

    accuracy                           0.60       149
   macro avg       0.47      0.50      0.39       149
weighted avg       0.50      0.60      0.46       149

[[ 1 58]
 [ 2 88]]
(0.49606202690692897, 0.5973154362416108, 0.4632340260016659, None)


In [43]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       1.00      0.02      0.03        59
           1       0.61      1.00      0.76        90

    accuracy                           0.61       149
   macro avg       0.80      0.51      0.39       149
weighted avg       0.76      0.61      0.47       149

[[ 1 58]
 [ 0 90]]
(0.7632867767095955, 0.610738255033557, 0.4700261312578723, None)


In [44]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.43      0.17      0.24        59
           1       0.61      0.86      0.71        90

    accuracy                           0.58       149
   macro avg       0.52      0.51      0.48       149
weighted avg       0.54      0.58      0.53       149

[[10 49]
 [13 77]]
(0.541289757805661, 0.5838926174496645, 0.5272275877121186, None)


In [45]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.43      0.63      0.51        59
           1       0.65      0.44      0.53        90

    accuracy                           0.52       149
   macro avg       0.54      0.54      0.52       149
weighted avg       0.56      0.52      0.52       149

[[37 22]
 [50 40]]
(0.5580971151142333, 0.5167785234899329, 0.5186075881991454, None)


In [46]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.40      0.58      0.47        59
           1       0.60      0.42      0.50        90

    accuracy                           0.48       149
   macro avg       0.50      0.50      0.48       149
weighted avg       0.52      0.48      0.49       149

[[34 25]
 [52 38]]
(0.5208811790675377, 0.48322147651006714, 0.4857372340280708, None)


## Test on stormfront

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_SF_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_SF_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.06      0.22      0.09       342
           1       0.87      0.59      0.70      2942

    accuracy                           0.55      3284
   macro avg       0.46      0.41      0.40      3284
weighted avg       0.78      0.55      0.64      3284

[[  75  267]
 [1202 1740]]
(0.7827950652852215, 0.5526796589524969, 0.6395917489204762, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.10      0.02      0.03       342
           1       0.90      0.98      0.94      2942

    accuracy                           0.88      3284
   macro avg       0.50      0.50      0.48      3284
weighted avg       0.81      0.88      0.84      3284

[[   7  335]
 [  61 2881]]
(0.8132605204079196, 0.879415347137637, 0.8418051315555103, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.06      0.01      0.01       342
           1       0.90      0.99      0.94      2942

    accuracy                           0.89      3284
   macro avg       0.48      0.50      0.48      3284
weighted avg       0.81      0.89      0.84      3284

[[   2  340]
 [  31 2911]]
(0.8084785271388775, 0.8870280146163215, 0.8433019276606549, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.16      0.01      0.03       342
           1       0.90      0.99      0.94      2942

    accuracy                           0.89      3284
   macro avg       0.53      0.50      0.48      3284
weighted avg       0.82      0.89      0.85      3284

[[   5  337]
 [  27 2915]]
(0.8192942589063529, 0.8891595615103532, 0.8459967113303832, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.15      0.08      0.11       342
           1       0.90      0.94      0.92      2942

    accuracy                           0.85      3284
   macro avg       0.52      0.51      0.51      3284
weighted avg       0.82      0.85      0.84      3284

[[  29  313]
 [ 169 2773]]
(0.8202485454974594, 0.8532277710109623, 0.835411226160446, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.12      0.30      0.17       342
           1       0.90      0.75      0.82      2942

    accuracy                           0.70      3284
   macro avg       0.51      0.53      0.50      3284
weighted avg       0.82      0.70      0.75      3284

[[ 103  239]
 [ 737 2205]]
(0.8210219371552853, 0.702801461632156, 0.7516694990848527, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.15      0.28      0.19       342
           1       0.91      0.81      0.86      2942

    accuracy                           0.75      3284
   macro avg       0.53      0.55      0.52      3284
weighted avg       0.83      0.75      0.79      3284

[[  96  246]
 [ 559 2383]]
(0.8272951427089998, 0.7548721071863581, 0.7864639469023617, None)


## Test on Reddit

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_RE_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_RE_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

         0.0       0.29      0.06      0.10      5813
         1.0       0.50      0.87      0.63      6188

    accuracy                           0.48     12001
   macro avg       0.39      0.46      0.36     12001
weighted avg       0.40      0.48      0.37     12001

[[ 337 5476]
 [ 814 5374]]
(0.39720817460067936, 0.4758770102491459, 0.3721480431404644, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

         0.0       0.53      0.03      0.06      5813
         1.0       0.52      0.97      0.68      6188

    accuracy                           0.52     12001
   macro avg       0.52      0.50      0.37     12001
weighted avg       0.52      0.52      0.38     12001

[[ 188 5625]
 [ 165 6023]]
(0.52458912624845, 0.5175402049829181, 0.37777674387337706, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

         0.0       0.62      0.04      0.07      5813
         1.0       0.52      0.98      0.68      6188

    accuracy                           0.52     12001
   macro avg       0.57      0.51      0.38     12001
weighted avg       0.57      0.52      0.39     12001

[[ 230 5583]
 [ 143 6045]]
(0.5667322237359137, 0.522873093908841, 0.3859229771013693, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

         0.0       0.23      0.00      0.00      5813
         1.0       0.51      0.99      0.68      6188

    accuracy                           0.51     12001
   macro avg       0.37      0.50      0.34     12001
weighted avg       0.37      0.51      0.35     12001

[[   9 5804]
 [  31 6157]]
(0.3744052104358469, 0.5137905174568786, 0.35133756220002293, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

         0.0       0.56      0.10      0.17      5813
         1.0       0.52      0.92      0.67      6188

    accuracy                           0.53     12001
   macro avg       0.54      0.51      0.42     12001
weighted avg       0.54      0.53      0.43     12001

[[ 590 5223]
 [ 467 5721]]
(0.539914297009106, 0.5258728439296725, 0.42756823304834535, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

         0.0       0.49      0.32      0.39      5813
         1.0       0.52      0.69      0.59      6188

    accuracy                           0.51     12001
   macro avg       0.50      0.50      0.49     12001
weighted avg       0.50      0.51      0.49     12001

[[1861 3952]
 [1939 4249]]
(0.5043654503221132, 0.5091242396466961, 0.4920649819600757, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

         0.0       0.55      0.37      0.45      5813
         1.0       0.55      0.71      0.62      6188

    accuracy                           0.55     12001
   macro avg       0.55      0.54      0.53     12001
weighted avg       0.55      0.55      0.54     12001

[[2176 3637]
 [1766 4422]]
(0.5503021161043179, 0.5497875177068577, 0.536174626515643, None)


## Test on Davidson

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_DAV_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_DAV_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.06      0.38      0.10       427
           1       0.94      0.61      0.74      7008

    accuracy                           0.60      7435
   macro avg       0.50      0.50      0.42      7435
weighted avg       0.89      0.60      0.71      7435

[[ 161  266]
 [2712 4296]]
(0.8908282267189142, 0.5994620040349697, 0.7055651811170124, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.06      0.04      0.04       427
           1       0.94      0.96      0.95      7008

    accuracy                           0.91      7435
   macro avg       0.50      0.50      0.50      7435
weighted avg       0.89      0.91      0.90      7435

[[  15  412]
 [ 248 6760]]
(0.8916980086284868, 0.9112306657700068, 0.9011946012845574, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.13      0.01      0.03       427
           1       0.94      0.99      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.54      0.50      0.50      7435
weighted avg       0.90      0.94      0.91      7435

[[   6  421]
 [  40 6968]]
(0.8963555792537543, 0.9379959650302623, 0.9138443708973527, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.08      0.01      0.02       427
           1       0.94      0.99      0.97      7008

    accuracy                           0.93      7435
   macro avg       0.51      0.50      0.49      7435
weighted avg       0.89      0.93      0.91      7435

[[   6  421]
 [  73 6935]]
(0.89298549863577, 0.9335574983187626, 0.9115145405780183, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.06      0.10      0.08       427
           1       0.94      0.91      0.93      7008

    accuracy                           0.86      7435
   macro avg       0.50      0.51      0.50      7435
weighted avg       0.89      0.86      0.88      7435

[[  43  384]
 [ 622 6386]]
(0.892819229246436, 0.864694014794889, 0.8782702657585698, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.06      0.23      0.09       427
           1       0.94      0.78      0.85      7008

    accuracy                           0.74      7435
   macro avg       0.50      0.50      0.47      7435
weighted avg       0.89      0.74      0.81      7435

[[  98  329]
 [1570 5438]]
(0.8921708220069388, 0.744586415601883, 0.8078293619139486, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.05      0.31      0.08       427
           1       0.94      0.62      0.75      7008

    accuracy                           0.60      7435
   macro avg       0.49      0.46      0.41      7435
weighted avg       0.89      0.60      0.71      7435

[[ 131  296]
 [2656 4352]]
(0.8852425132962238, 0.6029589778076665, 0.7085354933073278, None)


## Test on Waseem

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_WS_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_WS_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.34      0.87      0.49       898
           1       0.89      0.39      0.54      2500

    accuracy                           0.52      3398
   macro avg       0.62      0.63      0.51      3398
weighted avg       0.75      0.52      0.53      3398

[[ 781  117]
 [1527  973]]
(0.7461812810862835, 0.5161859917598587, 0.5275659538009804, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.79      0.36      0.49       898
           1       0.81      0.97      0.88      2500

    accuracy                           0.81      3398
   macro avg       0.80      0.66      0.69      3398
weighted avg       0.80      0.81      0.78      3398

[[ 319  579]
 [  83 2417]]
(0.803251279442217, 0.8051795173631549, 0.7768047898976187, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.89      0.47      0.61       898
           1       0.84      0.98      0.90      2500

    accuracy                           0.84      3398
   macro avg       0.86      0.72      0.76      3398
weighted avg       0.85      0.84      0.83      3398

[[ 419  479]
 [  52 2448]]
(0.8504225372525037, 0.8437316068275457, 0.8255086939480222, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.84      0.56      0.67       898
           1       0.86      0.96      0.91      2500

    accuracy                           0.86      3398
   macro avg       0.85      0.76      0.79      3398
weighted avg       0.85      0.86      0.84      3398

[[ 499  399]
 [  92 2408]]
(0.8542814060814637, 0.855503237198352, 0.8447865973412121, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.81      0.63      0.71       898
           1       0.88      0.95      0.91      2500

    accuracy                           0.86      3398
   macro avg       0.84      0.79      0.81      3398
weighted avg       0.86      0.86      0.86      3398

[[ 568  330]
 [ 133 2367]]
(0.8598375384851972, 0.8637433784579164, 0.8579323389860439, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.66      0.68      0.67       898
           1       0.88      0.88      0.88      2500

    accuracy                           0.82      3398
   macro avg       0.77      0.78      0.78      3398
weighted avg       0.83      0.82      0.83      3398

[[ 612  286]
 [ 311 2189]]
(0.8259372365350198, 0.8243084167157151, 0.8250729908958947, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.76      0.66      0.70       898
           1       0.88      0.92      0.90      2500

    accuracy                           0.85      3398
   macro avg       0.82      0.79      0.80      3398
weighted avg       0.85      0.85      0.85      3398

[[ 591  307]
 [ 189 2311]]
(0.8496895152072939, 0.8540317834020011, 0.8505821216132166, None)
